In [1]:
# WITHOUT PADDING
import torch
sample = 0.5

# WE ARE ON DATALOADER STAGE
arr1 = torch.tensor([2,4,6,8,10])#.unsqueeze(-1)
num_sample = int(len(arr1) * sample)

# Index for shuffle and revert
noise = torch.rand(arr1.shape)
shuffle = torch.argsort(noise, dim=0)
revert = torch.argsort(shuffle, dim=0)

# Get keep value
idx_keep = shuffle[:num_sample]
masked_input = torch.gather(arr1, index=idx_keep, dim=0)
print("ids_keep:", idx_keep)
print("masked_input:", masked_input) # Order of masked input does not matter since we already pos_encoded the sequence
print("_"*100)

# Revert
final_input = torch.cat([masked_input, torch.zeros(len(arr1) - num_sample)])
print("revert:", revert)
print("final_input:", final_input)
torch.gather(final_input, index=revert, dim=0)

ids_keep: tensor([2, 4])
masked_input: tensor([ 6, 10])
____________________________________________________________________________________________________
revert: tensor([4, 2, 0, 3, 1])
final_input: tensor([ 6., 10.,  0.,  0.,  0.])


tensor([ 0.,  0.,  6.,  0., 10.])

In [2]:
# WITH PADDING
max_seq_len = 10
mask_value = 0
padding_idx = max_seq_len

sample = 0.5
arr1 = torch.tensor([2,4,6,8,10])#.unsqueeze(-1)
num_sample = int(len(arr1) * sample)

# Index for shuffle and revert
noise = torch.rand(arr1.shape)
shuffle = torch.argsort(noise, dim=0)
revert = torch.argsort(shuffle, dim=0)

# Get keep value
idx_keep = shuffle[:num_sample]
masked_input = torch.gather(arr1, index=idx_keep, dim=0)
print("ids_keep:", idx_keep)
print("masked_input:", masked_input) # Order of masked input does not matter since we already pos_encoded the sequence

# Padding
padded_input = torch.concat([masked_input, torch.zeros(max_seq_len - len(masked_input))+mask_value])
print("padded_input:", padded_input)
print("_"*100)

# Revert
revert = torch.cat([revert, torch.zeros(max_seq_len-len(revert)) + (max_seq_len-1)]).to(torch.int64)
print("revert:", revert)
res = torch.gather(padded_input, index=revert, dim=0)
res

ids_keep: tensor([4, 0])
masked_input: tensor([10,  2])
padded_input: tensor([10.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
____________________________________________________________________________________________________
revert: tensor([1, 2, 4, 3, 0, 9, 9, 9, 9, 9])


tensor([ 2.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.])

In [3]:
# MULTIPLE DIMENSTIONS
# WITH PADDING
mask_value = 0

sample = 0.9
arr1 = torch.tensor([2,4])
arr2 = torch.tensor([2,4,6])
arr3 = torch.tensor([2,4,6,8])
arr4 = torch.tensor([2,4,6,8,10])

arr_li = [arr1, arr2, arr3, arr4]
revert_li = []
masked_input_li = []
max_seq_len = 0

for i in range(len(arr_li)):
    num_sample = int(len(arr_li[i]) * sample)

    # Index for shuffle and revert
    noise = torch.rand(arr_li[i].shape)
    shuffle = torch.argsort(noise, dim=0)
    revert = torch.argsort(shuffle, dim=0)
    max_seq_len = len(revert) if len(revert)>max_seq_len else max_seq_len
    revert_li.append(revert)
    

    # Get keep value
    idx_keep = shuffle[:num_sample]
    masked_input = torch.gather(arr_li[i], index=idx_keep, dim=0)
    masked_input_li.append(masked_input)
    print("ids_keep:", idx_keep)
    print("masked_input:", masked_input) # Order of masked input does not matter since we already pos_encoded the sequence
    print("-"*50)

# Padding
padded_input = torch.nn.utils.rnn.pad_sequence(masked_input_li, batch_first=True)
revert = torch.nn.utils.rnn.pad_sequence(revert_li, batch_first=True, padding_value=max_seq_len-1)
print("padded_input:\n", padded_input)
print("revert:\n", revert)
print("_"*100)

# Revert
print("max_seq_len:", max_seq_len)
filled_input = torch.concat([padded_input, torch.zeros(padded_input.shape[0], (max_seq_len - padded_input.shape[1]+1))], dim=1)
print("filled_input:\n", filled_input)
torch.gather(filled_input, index=revert, dim=-1)

ids_keep: tensor([1])
masked_input: tensor([4])
--------------------------------------------------
ids_keep: tensor([0, 1])
masked_input: tensor([2, 4])
--------------------------------------------------
ids_keep: tensor([3, 1, 2])
masked_input: tensor([8, 4, 6])
--------------------------------------------------
ids_keep: tensor([1, 0, 3, 2])
masked_input: tensor([4, 2, 8, 6])
--------------------------------------------------
padded_input:
 tensor([[4, 0, 0, 0],
        [2, 4, 0, 0],
        [8, 4, 6, 0],
        [4, 2, 8, 6]])
revert:
 tensor([[1, 0, 4, 4, 4],
        [0, 1, 2, 4, 4],
        [3, 1, 2, 0, 4],
        [1, 0, 3, 2, 4]])
____________________________________________________________________________________________________
max_seq_len: 5
filled_input:
 tensor([[4., 0., 0., 0., 0., 0.],
        [2., 4., 0., 0., 0., 0.],
        [8., 4., 6., 0., 0., 0.],
        [4., 2., 8., 6., 0., 0.]])


tensor([[0., 4., 0., 0., 0.],
        [2., 4., 0., 0., 0.],
        [0., 4., 6., 8., 0.],
        [2., 4., 6., 8., 0.]])

In [ ]:
import torch
# MULTIPLE DIMENSTIONS
# WITH PADDING
mask_value = 0

sample = 0.5
arr1 = torch.tensor([2,4])
arr2 = torch.tensor([2,4,6])
arr3 = torch.tensor([2,4,6,8])
arr4 = torch.tensor([2,4,6,8,10])

arr_li = [arr1, arr2, arr3, arr4]
revert_li = []
keep_li = []
max_seq_len = 0

for i in range(len(arr_li)):
    num_sample = int(len(arr_li[i]) * sample)

    # Index for shuffle and revert
    noise = torch.rand(arr_li[i].shape)
    shuffle = torch.argsort(noise, dim=0)
    revert = torch.argsort(shuffle, dim=0)
    max_seq_len = len(revert) if len(revert)>max_seq_len else max_seq_len
    revert_li.append(revert)

    # Get keep value
    idx_keep = shuffle[:num_sample]
    keep_li.append(idx_keep)

# Padding
arr = torch.nn.utils.rnn.pad_sequence(arr_li, batch_first=True)
revert = torch.nn.utils.rnn.pad_sequence(revert_li, batch_first=True, padding_value=max_seq_len-1)
keep = torch.nn.utils.rnn.pad_sequence(keep_li, batch_first=True, padding_value=max_seq_len-1)
print("arr:\n", arr)
print("revert:\n", revert)
print("keep:\n", keep)
print("_"*100)

# Select
# Todo: Linear trnasform & pos_encoding
masked_input = torch.gather(arr, index=keep, dim=-1)
print("masked_input:\n", masked_input)

# Revert
filled_input = torch.concat([masked_input, torch.zeros(masked_input.shape[0], (max_seq_len - masked_input.shape[1]+1))], dim=1)
print("filled_input:\n", filled_input)
print()
torch.gather(filled_input, index=revert, dim=-1)